# NY Times Collector for OpenTLDR (Update)
This notebook pulls raw data from the internet and formats it for OpenTLDR ingest as a .json file.
The output json file can be shared with multiple OpenTLDR instances as a file system or S3 bucket.

In [ ]:

import os
import logging

from dotenv import load_dotenv
load_dotenv()

from RssFeed import RssFeed
from RepoWriter import RepoWriter
from HtmlScrape import url_to_filename


In [ ]:
# Parameters

# The path where this agent should write the .json files for content
live_data_repo_path = os.getenv('LIVE_DATA_REPO_PATH')

# The path where this agent should write full source files temporarily
download_media = False  # Note NY Times requires JavaScript to be enabled to load
media_cache_path = os.getenv('MEDIA_CACHE_PATH')

# Name of Source to use in OpenTLDR objects
rss_source = "nytimes"
organize_by_source = True

# URLs for the feeds to scrape for this Source
rss_feed_urls = [
    "https://rss.nytimes.com/services/xml/rss/nyt/World.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Africa.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Americas.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Europe.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/MiddleEast.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/US.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Education.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/NYRegion.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Business.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/EnergyEnvironment.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/SmallBusiness.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Economy.xml",
#   "https://rss.nytimes.com/services/xml/rss/nyt/Dealbook.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/MediaandAdvertising.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/YourMoney.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/PersonalTech.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Sports.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Baseball.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/CollegeBasketball.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/CollegeFootball.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Golf.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Hockey.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/ProBasketball.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/ProFootball.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Soccer.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Tennis.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Science.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Climate.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/Space.xml",
#    "https://rss.nytimes.com/services/xml/rss/nyt/Arts.xml",
]

# Filters - excludes some content
date:str = None
days_history = 9999  # it seems like forever...

# Some sites verify recent user_agent configurations
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"

# Logging level ranges are (from least to most verbose): ERROR, WARN, INFO, DEBUG
logging_level = logging.INFO

# level of unnecessary output
verbose = True


## Process Parameters
Parameters are passed in as strings that sometimes need to be converted to other object format.


In [ ]:
from datetime import datetime, timedelta
import pytz

# Default to today
today = datetime.now()
if date is not None: 
    today = datetime.strptime(date,"%Y-%m-%d")

# defaults to previous 1 day
last_read = today - timedelta(days=days_history)

print("Today is {}, will atttempt to download content since {}.".format(today.strftime('%Y-%m-%d'),last_read.strftime('%Y-%m-%d')))

In [ ]:
# Verify where the .json files should be stored.
if live_data_repo_path is None:
    live_data_repo_path = os.path.join("..","Data","Live")

if organize_by_source:
    live_data_repo_path= os.path.join(live_data_repo_path,rss_source)

print ("Writing Content to: {}".format(live_data_repo_path))

# Verify where the media files should be stored.
if media_cache_path is None and download_media:
    media_cache_path = os.path.join('..','Data','MediaCache')

if download_media:
    print ("Cacheing Media files to: {}".format(media_cache_path))

In [ ]:
# Ensure that these paths exist and create them if they do not

if not os.path.exists(live_data_repo_path):
    print ("Creating directory: {}".format(live_data_repo_path))
    os.makedirs(live_data_repo_path)

if download_media and not os.path.exists(media_cache_path):
    print ("Creating directory: {}".format(media_cache_path))
    os.makedirs(media_cache_path)

# Processing NY Times Feed

In [ ]:
reader=RssFeed(rss_source, user_agent=user_agent)
reader.set_filter(today=today, earliest=last_read)
reader.set_log_level(logging_level)
#reader.archive = os.path.join(live_data_repo_path,"rss_archive",datetime.now().strftime("%Y-%m-%d"))

writer=RepoWriter(live_data_repo_path)
writer.set_log_level(logging_level)

for feed in rss_feed_urls:

    try:
        for entry in reader.fetch_feed(feed, type = "news"):

            # Clean up source-specific issues
            source_html_url = entry['url']

            # Write out a content node 
            repo_uid = writer.write_content(entry)

            # Download Full Media into raw folder
            if repo_uid is not None and download_media:
                try:
                    if not os.path.exists(media_cache_path):
                        os.makedirs(media_cache_path)

                    for m in entry['metadata']['media']:
                        media_url:str = m
                        media_filename:str = writer.url_to_filename(m)
                        media_full_path = os.path.join(media_cache_path,media_filename)

                        if not os.path.exists(media_full_path):                
                            print ("Downloading media file {} for {}".format(media_full_path,repo_uid))
                            writer.http_copy(media_url, media_full_path)
                        else:
                            print ("Already have media file {} for {}".format(media_full_path,repo_uid))

                    html_media_path = os.path.join(media_cache_path,"{}.html".format(repo_uid))
                    writer.http_copy(source_html_url, html_media_path)
                    
                except Exception as e:
                    print ("Failed to download HTML file for {}: {}".format(repo_uid, e))
    
    except Exception as e:
        print ("RSS Feed {} had error: {}".format(feed, e))